<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_SMARTCITY_DT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mistralai -q
!pip install colab_env -q

In [2]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env

# Ensure colab-env and mistralai are installed and API key is set
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    # !pip install colab-env -q # Uncomment if running in Colab and colab_env is not found
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic models for Smart City outputs (Adapted from AOCC outputs)
class CityOperationStatus(BaseModel):
    zone_id: str
    status: str # e.g., "Operational", "Disrupted", "Under Maintenance"
    incident_type: str = None
    affected_services: list = None
    estimated_recovery_time: str = None
    flagged_issues: dict = None

class InfrastructureHealthReport(BaseModel):
    infrastructure_id: str # e.g., "Bridge-001", "Road_Segment_A", "Building-123"
    health_status: str # e.g., "Good", "Minor Degradation", "Critical", "Under Repair"
    last_inspection_date: str
    next_inspection_due: str
    issues_reported: list
    recommended_repairs: list

print("Creating AI agents for Smart City Domain...")

# Agent Definitions (Smart City Domain)

# 1. Urban Planning & Development Agent (Adapted from Flight Planning Agent)
urban_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for strategic urban planning, simulating development impacts, and managing city infrastructure projects.",
    name="urban-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "design_urban_plan",
                "description": "Design an urban development plan based on area, type of development (e.g., 'residential', 'commercial'), and optional specific requirements.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "area_id": {"type": "string", "description": "Identifier for the urban area."},
                        "development_type": {"type": "string", "description": "Type of development (e.g., 'residential', 'commercial', 'mixed-use')."},
                        "specific_requirements": {"type": "string", "description": "Optional: Specific requirements for the plan (e.g., 'green spaces', 'high-density')."}
                    },
                    "required": ["area_id", "development_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_project_schedule",
                "description": "Log the scheduled milestones and completion times for an urban development project.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "project_id": {"type": "string", "description": "Unique identifier for the project."},
                        "scheduled_start": {"type": "string", "description": "Scheduled start time."},
                        "scheduled_completion": {"type": "string", "description": "Scheduled completion time."}
                    },
                    "required": ["project_id", "scheduled_start", "scheduled_completion"]
                }
            }
        }
    ]
)
print(f"Urban Planning & Development Agent '{urban_planning_agent.name}' created with ID: {urban_planning_agent.id}")

# 2. Urban Traffic Management Agent (Adapted from Air Traffic Control Liaison Agent)
urban_traffic_management_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for monitoring and controlling urban traffic flow, predicting congestion, and optimizing signal timings.",
    name="urban-traffic-agent",
    tools=[
        {"type": "web_search"}, # For searching external data like public transport alerts or major event schedules.
        {
            "type": "function",
            "function": {
                "name": "predict_traffic_congestion",
                "description": "Predict the level of traffic congestion for a given area (e.g., 'intersection', 'road segment') and time window.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "area_id": {"type": "string", "description": "Identifier for the urban area (e.g., 'MainSt_ElmAve', 'Highway_40_Segment')."},
                        "time_window": {"type": "string", "description": "Time period for congestion prediction."}
                    },
                    "required": ["area_id", "time_window"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "optimize_signal_timing",
                "description": "Adjust the green light duration and cycle for a specific traffic signal or intersection to improve traffic flow.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "intersection_id": {"type": "string", "description": "Identifier of the intersection."},
                        "optimization_goal": {"type": "string", "description": "Goal for optimization (e.g., 'reduce wait time', 'prioritize public transport')."}
                    },
                    "required": ["intersection_id", "optimization_goal"]
                }
            }
        }
    ]
)
print(f"Urban Traffic Management Agent '{urban_traffic_management_agent.name}' created with ID: {urban_traffic_management_agent.id}")

# 3. Urban Disruption Response Agent (Adapted from Disruption Management Agent)
urban_disruption_response_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="urban-disruption-agent",
    description="Agent to predict and manage urban disruptions (e.g., traffic accidents, public utility failures, extreme weather impacts) and propose recovery strategies.",
    instructions="Predict disruption impact and propose effective recovery strategies for urban incidents.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "city_operation_status",
                "schema": CityOperationStatus.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "predict_disruption_impact",
                "description": "Predict the impact level of a disruption (e.g., 'power outage', 'road closure', 'flood') for a given city zone or service.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "zone_id": {"type": "string", "description": "Identifier of the affected city zone or area."},
                        "disruption_type": {"type": "string", "description": "The type of disruption (e.g., 'weather', 'utility_failure', 'accident')."}
                    },
                    "required": ["zone_id", "disruption_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "propose_recovery_plan",
                "description": "Suggest a recovery plan for a disrupted urban service or affected city area.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "zone_id": {"type": "string", "description": "Identifier of the city zone requiring recovery."},
                        "disruption_details": {"type": "string", "description": "Details of the disruption and its current state."}
                    },
                    "required": ["zone_id", "disruption_details"]
                }
            }
        }
    ]
)
print(f"Urban Disruption Response Agent '{urban_disruption_response_agent.name}' created with ID: {urban_disruption_response_agent.id}")

# 4. City Resource Allocation Agent (Adapted from Crew Scheduling Agent)
city_resource_allocation_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for optimizing allocation of city resources (e.g., emergency services, public works teams) and managing their deployment.",
    name="city-resource-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "assign_resources_to_task",
                "description": "Assign specific teams or vehicles to a city task or incident.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {"type": "string", "description": "Identifier of the task or incident."},
                        "team_ids": {"type": "array", "items": {"type": "string"}, "description": "List of resource team identifiers."},
                        "vehicle_ids": {"type": "array", "items": {"type": "string"}, "description": "List of vehicle identifiers."}
                    },
                    "required": ["task_id", "team_ids"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "simulate_resource_deployment",
                "description": "Simulate the impact of resource reassignments on response times and service availability.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "resource_id": {"type": "string", "description": "Identifier of the resource (team or vehicle)."},
                        "new_assignment": {"type": "string", "description": "Description of the new assignment (e.g., 'deploy to Zone C', 'reassign to incident XYZ')."}
                    },
                    "required": ["resource_id", "new_assignment"]
                }
            }
        }
    ]
)
print(f"City Resource Allocation Agent '{city_resource_allocation_agent.name}' created with ID: {city_resource_allocation_agent.id}")

# 5. Infrastructure Maintenance Agent (Adapted from Aircraft Maintenance Agent)
infrastructure_maintenance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing urban infrastructure maintenance schedules (e.g., roads, bridges, public buildings), tracking material availability, and logging maintenance records.",
    name="infrastructure-maintenance-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "schedule_maintenance",
                "description": "Schedule a maintenance event for a specific piece of urban infrastructure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "infrastructure_id": {"type": "string", "description": "Identifier of the infrastructure (e.g., 'Bridge-001', 'Road_Segment_A')."},
                        "maintenance_type": {"type": "string", "description": "Type of maintenance (e.g., 'inspection', 'repair', 'upgrade')."},
                        "preferred_date": {"type": "string", "description": "Preferred date for maintenance."}
                    },
                    "required": ["infrastructure_id", "maintenance_type", "preferred_date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "check_materials_inventory",
                "description": "Check the availability and location of specific urban construction materials in inventory.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "material_name": {"type": "string", "description": "Name of the construction material (e.g., 'asphalt', 'steel beams')."}
                    },
                    "required": ["material_name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_maintenance_record",
                "description": "Log the completion of a maintenance task for urban infrastructure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "infrastructure_id": {"type": "string", "description": "Identifier of the infrastructure."},
                        "service_description": {"type": "string", "description": "Description of the maintenance performed."},
                        "date_completed": {"type": "string", "description": "Date of completion (ISO format)."}
                    },
                    "required": ["infrastructure_id", "service_description", "date_completed"]
                }
            }
        }
    ]
)
print(f"Infrastructure Maintenance Agent '{infrastructure_maintenance_agent.name}' created with ID: {infrastructure_maintenance_agent.id}")

# 6. Urban Data Analytics Agent (Adapted from Operational Data Analysis Agent)
urban_data_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for accessing, analyzing, and interpreting diverse urban operational data (e.g., sensor data, utility consumption, incident logs).",
    name="urban-data-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_city_data",
                "description": "Retrieve raw or processed operational data from a specific city system or area.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_source": {"type": "string", "description": "Source of the data (e.g., 'traffic sensors', 'smart meters', 'weather station')."},
                        "data_type": {"type": "string", "description": "Optional: Specific type of data to retrieve (e.g., 'flow rates', 'temperature', 'occupancy')."}
                    },
                    "required": ["data_source"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_urban_efficiency",
                "description": "Perform statistical analysis on urban operational data to determine efficiency (e.g., energy consumption per capita, traffic flow rates).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_json": {"type": "string", "description": "JSON string of numerical urban operational data."},
                        "comparison_data_json": {"type": "string", "description": "Optional: JSON string of comparison data for benchmarking."}
                    },
                    "required": ["data_json"]
                }
            }
        }
    ]
)
print(f"Urban Data Analytics Agent '{urban_data_analysis_agent.name}' created with ID: {urban_data_analysis_agent.id}")

# 7. City Compliance & Safety Agent (Adapted from Compliance & Safety Agent)
city_compliance_safety_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for ensuring adherence to city regulations (e.g., building codes, environmental standards, public safety protocols).",
    name="city-compliance-safety-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_city_regulation",
                "description": "Check a specific aspect of urban operations against relevant city or regional regulations.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aspect": {"type": "string", "description": "Aspect to check (e.g., 'building code', 'noise ordinance', 'environmental emissions')."},
                        "region": {"type": "string", "description": "Regulatory region (e.g., 'Downtown', 'North Zone', 'City-wide')."}
                    },
                    "required": ["aspect", "region"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "submit_public_safety_report",
                "description": "Simulate submission of a public safety report for an incident or observation.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "incident_type": {"type": "string", "description": "Type of incident (e.g., 'public disturbance', 'infrastructure damage', 'environmental spill')."},
                        "summary": {"type": "string", "description": "Summary of the incident or observation."}
                    },
                    "required": ["incident_type", "summary"]
                }
            }
        }
    ]
)
print(f"City Compliance & Safety Agent '{city_compliance_safety_agent.name}' created with ID: {city_compliance_safety_agent.id}")

# 8. Urban Incident Response Agent (Adapted from Incident Response Agent)
urban_incident_response_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for analyzing urban incidents (e.g., major traffic accidents, utility failures), identifying root causes, and predicting cascading effects on city services.",
    name="urban-incident-response-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "analyze_incident",
                "description": "Analyze existing urban operational data to infer or confirm the root cause of an incident.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "incident_id": {"type": "string", "description": "Identifier of the incident."},
                        "operational_data_json": {"type": "string", "description": "JSON string of relevant urban operational data (e.g., sensor readings, traffic logs, utility metrics)."}
                    },
                    "required": ["incident_id", "operational_data_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "predict_cascading_effects",
                "description": "Predict potential cascading effects of an urban incident on other city services or areas.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "incident_id": {"type": "string", "description": "Identifier of the incident."},
                        "affected_services_or_zones": {"type": "array", "items": {"type": "string"}, "description": "List of potentially affected city services or zones."}
                    },
                    "required": ["incident_id", "affected_services_or_zones"]
                }
            }
        }
    ]
)
print(f"Urban Incident Response Agent '{urban_incident_response_agent.name}' created with ID: {urban_incident_response_agent.id}")

# 9. Public Works Coordination Agent (Adapted from Ground Operations Coordination Agent)
public_works_coordination_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for coordinating with public works teams, waste management, and other essential city services.",
    name="public-works-agent",
    tools=[
        {"type": "web_search"}, # For looking up external city service provider contact info or specific municipal guidelines.
        {
            "type": "function",
            "function": {
                "name": "track_city_activity",
                "description": "Track the completion status of a specific public works activity for a given city area or asset.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "area_id": {"type": "string", "description": "Identifier of the city area or asset."},
                        "activity_name": {"type": "string", "description": "Name of the public works activity (e.g., 'street cleaning', 'pothole repair')."}
                    },
                    "required": ["area_id", "activity_name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "request_city_service",
                "description": "Request a specific city service (e.g., 'road repair crew dispatch', 'public waste collection', 'tree trimming').",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "service_area": {"type": "string", "description": "Identifier of the area requesting service."},
                        "service_description": {"type": "string", "description": "Detailed description of the requested service."}
                    },
                    "required": ["service_area", "service_description"]
                }
            }
        }
    ]
)
print(f"Public Works Coordination Agent '{public_works_coordination_agent.name}' created with ID: {public_works_coordination_agent.id}")

# NEW AGENTS ADDED (Adapted from the NEW AGENTS ADDED section in reference)

# 10. Citizen Services Agent (Adapted from Passenger Coordinator Agent)
citizen_services_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing citizen requests, re-routing services during disruptions, and sending public notifications.",
    name="citizen-services-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_citizen_request_status",
                "description": "Retrieves the status and details for a given citizen service request.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "request_id": {"type": "string", "description": "Unique identifier for the citizen service request."}
                    },
                    "required": ["request_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "reroute_service_request",
                "description": "Reroutes a citizen service request from one department or team to an alternative one.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "request_id": {"type": "string", "description": "Unique identifier for the service request."},
                        "current_department": {"type": "string", "description": "Current department handling the request."},
                        "new_department": {"type": "string", "description": "Identifier of the new department to route the request to."}
                    },
                    "required": ["request_id", "current_department", "new_department"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "send_public_notification",
                "description": "Sends an update or notification to a specific citizen or city zone.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "recipient_id": {"type": "string", "description": "Unique identifier for the citizen or zone."},
                        "message_content": {"type": "string", "description": "The content of the message."},
                        "contact_method": {"type": "string", "description": "Method of contact (e.g., 'email', 'SMS', 'public_announcement')."}
                    },
                    "required": ["recipient_id", "message_content", "contact_method"]
                }
            }
        }
    ]
)
print(f"Citizen Services Agent '{citizen_services_agent.name}' created with ID: {citizen_services_agent.id}")

# 11. Emergency Services Coordinator Agent (Adapted from Crew Coordinator Agent)
emergency_services_coordinator_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing emergency service assignments (police, fire, EMS), ensuring compliance with response protocols, and coordinating deployment.",
    name="emergency-services-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_response_legality",
                "description": "Checks if an emergency response unit is compliant with deployment regulations (e.g., staffing, equipment, duty limits).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "unit_id": {"type": "string", "description": "Identifier of the emergency unit (e.g., 'FireTruck-001', 'PoliceCar-A')."}
                    },
                    "required": ["unit_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "find_available_unit",
                "description": "Searches for available emergency units with specific roles and qualifications within a given area.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "role": {"type": "string", "description": "Role of the unit (e.g., 'Fire', 'Police', 'EMS')."},
                        "required_qualifications": {"type": "array", "items": {"type": "string"}, "description": "List of required qualifications (e.g., 'HAZMAT certified', 'paramedic')."},
                        "area": {"type": "string", "description": "Optional: Area to search for available units."}
                    },
                    "required": ["role", "required_qualifications"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "assign_unit_to_incident",
                "description": "Assigns an emergency unit to a specific incident, updating their status and dispatch information.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "unit_id": {"type": "string", "description": "Identifier of the emergency unit."},
                        "incident_details": {"type": "string", "description": "Description of the incident or assignment."}
                    },
                    "required": ["unit_id", "incident_details"]
                }
            }
        }
    ]
)
print(f"Emergency Services Coordinator Agent '{emergency_services_coordinator_agent.name}' created with ID: {emergency_services_coordinator_agent.id}")

# 12. Smart Grid Operations Agent (Adapted from Flight Dispatcher Agent)
smart_grid_operations_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for real-time energy grid monitoring, demand forecasting, renewable energy integration, and managing grid stability.",
    name="smart-grid-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "monitor_grid_status",
                "description": "Monitors the current power flow, stability, and status of various energy grid components (e.g., substations, smart meters).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "component_id": {"type": "string", "description": "Identifier of the grid component to monitor (e.g., 'Substation-A', 'Zone-B')."}
                    },
                    "required": ["component_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_weather_impact_on_energy",
                "description": "Analyzes weather conditions and assesses potential impact on renewable energy generation (solar, wind) or energy demand.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "area": {"type": "string", "description": "Description of the city area for weather analysis."},
                        "time_window": {"type": "string", "description": "Time period for weather analysis and energy impact assessment."}
                    },
                    "required": ["area", "time_window"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "calculate_energy_distribution",
                "description": "Calculates optimal energy distribution paths and load balancing based on real-time demand, supply, and grid health.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "zone_id": {"type": "string", "description": "Identifier of the city zone for energy calculation."},
                        "current_demand_kwh": {"type": "number", "description": "Current energy demand in kWh."},
                        "available_supply_kwh": {"type": "number", "description": "Total available energy supply in kWh."},
                        "priority_services": {"type": "array", "items": {"type": "string"}, "description": "Optional: List of high-priority services to ensure power for."}
                    },
                    "required": ["zone_id", "current_demand_kwh", "available_supply_kwh"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "issue_grid_directive",
                "description": "Formally issues a directive to grid components (e.g., 'increase solar output', 'activate battery storage', 'shed load').",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "component_id": {"type": "string", "description": "Identifier of the grid component to direct."},
                        "directive_summary": {"type": "string", "description": "Summary of the directive to be issued."}
                    },
                    "required": ["component_id", "directive_summary"]
                }
            }
        }
    ]
)
print(f"Smart Grid Operations Agent '{smart_grid_operations_agent.name}' created with ID: {smart_grid_operations_agent.id}")

# 13. Urban Budget & Finance Agent (Adapted from Budget Finance Agent)
urban_budget_finance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for monitoring city operational budgets, tracking expenses, forecasting costs, and optimizing financial performance for urban projects and services.",
    name="urban-budget-finance-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_departmental_budget_status",
                "description": "Retrieves the current status of a specific city departmental budget for a given financial period.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "department": {"type": "string", "description": "The city department (e.g., 'Public Works', 'Parks and Recreation')."},
                        "period": {"type": "string", "description": "The financial period (e.g., 'Q3 2025', 'FY 2026')."}
                    },
                    "required": ["department", "period"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_city_expense",
                "description": "Logs an operational expense against a specific category, optionally linked to a city project or service.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expense_category": {"type": "string", "description": "The category of the expense (e.g., 'infrastructure maintenance', 'public safety')."},
                        "amount": {"type": "number", "description": "The amount of the expense."},
                        "project_id": {"type": "string", "description": "Optional: Identifier of the city project or service this expense relates to."}
                    },
                    "required": ["expense_category", "amount"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "forecast_city_costs",
                "description": "Forecasts future operational costs for a given cost type and time horizon for city services.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cost_type": {"type": "string", "description": "The type of cost to forecast (e.g., 'utility costs', 'payroll', 'material costs')."},
                        "time_horizon": {"type": "string", "description": "The period for the forecast (e.g., 'next quarter', 'next year')."}
                    },
                    "required": ["cost_type", "time_horizon"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_cost_efficiency",
                "description": "Analyzes city operational cost data to identify areas for efficiency improvement.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_json": {"type": "string", "description": "JSON string of numerical cost data for analysis."},
                        "benchmark_data_json": {"type": "string", "description": "Optional: JSON string of benchmark cost data for comparison."}
                    },
                    "required": ["data_json"]
                }
            }
        }
    ]
)
print(f"Urban Budget & Finance Agent '{urban_budget_finance_agent.name}' created with ID: {urban_budget_finance_agent.id}")

print("\nAll Smart City agents have been defined.")

# Mock functions for Smart City domain tools (Adapted from AOCC Mocks)

# Urban Planning & Development Agent Mocks
def design_urban_plan(area_id: str, development_type: str, specific_requirements: str = None):
    """MOCK function to design an urban plan."""
    print(f"\n [DEBUG] MOCK CALL: design_urban_plan Area: '{area_id}', Type: '{development_type}', Requirements: '{specific_requirements}'")
    if area_id == "Downtown" and development_type == "residential":
        return {"status": "success", "plan_id": "URBAN-001", "message": "Residential urban plan designed for Downtown."}
    return {"status": "success", "plan_id": "URBAN-GENERIC", "message": "Basic urban plan designed."}

def log_project_schedule(project_id: str, scheduled_start: str, scheduled_completion: str):
    """MOCK function to log project schedule."""
    print(f"\n [DEBUG] MOCK CALL: log_project_schedule Project: {project_id}, Start: {scheduled_start}, Completion: {scheduled_completion}")
    return {"status": "logged", "project_id": project_id, "message": "Project schedule logged successfully."}

# Urban Traffic Management Agent Mocks
def predict_traffic_congestion(area_id: str, time_window: str):
    """MOCK function to predict traffic congestion."""
    print(f"\n [DEBUG] MOCK CALL: predict_traffic_congestion Area: {area_id}, Time Window: {time_window}")
    if area_id == "MainSt_ElmAve" and "next 2 hours" in time_window:
        return {"status": "warning", "congestion_level": "High", "message": "High congestion predicted at Main St & Elm Ave."}
    return {"status": "low", "congestion_level": "Low", "message": "Low congestion expected."}

def optimize_signal_timing(intersection_id: str, optimization_goal: str):
    """MOCK function to optimize signal timing."""
    print(f"\n [DEBUG] MOCK CALL: optimize_signal_timing Intersection: {intersection_id}, Goal: '{optimization_goal}'")
    if intersection_id == "MainSt_ElmAve" and optimization_goal == "reduce wait time":
        return {"status": "optimized", "details": "Signal timings adjusted to reduce wait times by 15%.", "message": "Traffic signal optimized."}
    return {"status": "optimized", "message": "Signal optimization applied based on general rules."}

# Urban Disruption Response Agent Mocks
def predict_disruption_impact(zone_id: str, disruption_type: str):
    """MOCK function to predict disruption impact."""
    print(f"\n [DEBUG] MOCK CALL: predict_disruption_impact Zone: {zone_id}, Disruption: {disruption_type}")
    if disruption_type == "power outage" and zone_id == "Downtown":
        return {"status": "warning", "impact_level": "High", "affected_services": ["traffic lights", "public transit"], "message": "High impact predicted for Downtown due to power outage."}
    return {"status": "low_impact", "message": "Minor disruption, manageable."}

def propose_recovery_plan(zone_id: str, disruption_details: str):
    """MOCK function to propose recovery plan."""
    print(f"\n [DEBUG] MOCK CALL: propose_recovery_plan Zone: {zone_id}, Details: '{disruption_details}'")
    if "major road closure" in disruption_details:
        return {"status": "plan_proposed", "plan": "Reroute traffic, deploy signage, inform public.", "message": "Major road closure recovery plan proposed."}
    return {"status": "plan_proposed", "plan": "Minor service adjustment.", "message": "Minor recovery plan proposed."}

# City Resource Allocation Agent Mocks
def assign_resources_to_task(task_id: str, team_ids: list, vehicle_ids: list = None):
    """MOCK function to assign resources to task."""
    print(f"\n [DEBUG] MOCK CALL: assign_resources_to_task Task: {task_id}, Teams: {team_ids}, Vehicles: {vehicle_ids}")
    return {"status": "assigned", "task_id": task_id, "message": "Resources assigned successfully."}

def simulate_resource_deployment(resource_id: str, new_assignment: str):
    """MOCK function to simulate resource deployment."""
    print(f"\n [DEBUG] MOCK CALL: simulate_resource_deployment Resource: {resource_id}, New Assignment: '{new_assignment}'")
    if resource_id == "Team-PW-001" and "Zone C" in new_assignment:
        return {"status": "simulation_complete", "impact": "response time improved", "message": "Resource deployment simulated successfully."}
    return {"status": "simulation_complete", "impact": "neutral", "message": "Resource change simulated."}

# Infrastructure Maintenance Agent Mocks
def schedule_maintenance(infrastructure_id: str, maintenance_type: str, preferred_date: str):
    """MOCK function to schedule maintenance."""
    print(f"\n [DEBUG] MOCK CALL: schedule_maintenance Infrastructure: {infrastructure_id}, Type: {maintenance_type}, Date: {preferred_date}")
    return {"status": "scheduled", "infrastructure_id": infrastructure_id, "message": "Maintenance scheduled."}

def check_materials_inventory(material_name: str):
    """MOCK function to check materials inventory."""
    print(f"\n [DEBUG] MOCK CALL: check_materials_inventory Material: {material_name}")
    if material_name == "asphalt":
        return {"status": "in_stock", "quantity_tons": 500, "location": "City Depot A", "message": "Material in stock."}
    return {"status": "out_of_stock", "message": "Material not in stock."}

def log_maintenance_record(infrastructure_id: str, service_description: str, date_completed: str):
    """MOCK function to log maintenance record."""
    print(f"\n [DEBUG] MOCK CALL: log_maintenance_record Infrastructure: {infrastructure_id}, Service: '{service_description}', Date: {date_completed}")
    return {"status": "logged", "infrastructure_id": infrastructure_id, "message": "Maintenance record logged."}

# Urban Data Analytics Agent Mocks
def get_city_data(data_source: str, data_type: str = "general"):
    """MOCK function to retrieve city data."""
    print(f"\n [DEBUG] MOCK CALL: get_city_data Source: {data_source}, Type: {data_type}")
    if data_source == "traffic sensors" and data_type == "flow rates":
        return {"status": "success", "data": "Traffic flow data for Main St: [avg_speed: 40km/h, density: 0.7]", "message": "City data retrieved."}
    return {"status": "not_found", "message": "City data not found for specified source/type."}

def analyze_urban_efficiency(data_json: str, comparison_data_json: str = None):
    """MOCK function to analyze urban efficiency."""
    print(f"\n [DEBUG] MOCK CALL: analyze_urban_efficiency Data: {data_json}, Comparison: {comparison_data_json}")
    data = json.loads(data_json)
    if "energy_consumption" in data:
        avg_consumption = sum(data["energy_consumption"]) / len(data["energy_consumption"]) if data["energy_consumption"] else 0
        return {"status": "success", "average_consumption_kwh": avg_consumption, "message": "Energy efficiency analysis complete."}
    return {"status": "success", "message": "Efficiency analysis complete (generic)."}

# City Compliance & Safety Agent Mocks
def check_city_regulation(aspect: str, region: str):
    """MOCK function to check city regulation."""
    print(f"\n [DEBUG] MOCK CALL: check_city_regulation Aspect: {aspect}, Region: {region}")
    if aspect == "building code" and region == "Downtown":
        return {"status": "compliant", "details": "Compliant with Downtown building codes.", "message": "Regulation check complete."}
    return {"status": "non_compliant", "details": "Requires further review.", "message": "Potential non-compliance."}

def submit_public_safety_report(incident_type: str, summary: str):
    """MOCK function to submit public safety report."""
    print(f"\n [DEBUG] MOCK CALL: submit_public_safety_report Type: {incident_type}, Summary: '{summary}'")
    return {"status": "submitted", "report_id": "PSR-001", "message": "Public safety report submitted."}

# Urban Incident Response Agent Mocks
def analyze_incident(incident_id: str, operational_data_json: str):
    """MOCK function to analyze incident."""
    print(f"\n [DEBUG] MOCK CALL: analyze_incident Incident: {incident_id}, Data: {operational_data_json}")
    if incident_id == "UINC-005" and "traffic sensor anomaly" in operational_data_json:
        return {"status": "success", "root_cause": "Traffic sensor malfunction.", "confidence": 0.85, "message": "Incident analysis complete."}
    return {"status": "needs_more_data", "message": "Further data needed for root cause analysis."}

def predict_cascading_effects(incident_id: str, affected_services_or_zones: list):
    """MOCK function to predict cascading effects."""
    print(f"\n [DEBUG] MOCK CALL: predict_cascading_effects Incident: {incident_id}, Affected: {affected_services_or_zones}")
    if incident_id == "UINC-005" and "traffic lights" in affected_services_or_zones:
        return {"status": "warning", "cascading_impact": "Potential traffic light malfunctions, increased congestion.", "message": "Cascading effects predicted."}
    return {"status": "no_major_impact", "message": "No significant cascading effects predicted."}

# Public Works Coordination Agent Mocks
def track_city_activity(area_id: str, activity_name: str):
    """MOCK function to track city activity."""
    print(f"\n [DEBUG] MOCK CALL: track_city_activity Area: {area_id}, Activity: '{activity_name}'")
    if area_id == "Downtown" and activity_name == "street cleaning":
        return {"status": "completed", "completion_time": "2025-06-27T10:00:00Z", "message": "Street cleaning complete."}
    return {"status": "in_progress", "message": "Activity ongoing or not found."}

def request_city_service(service_area: str, service_description: str):
    """MOCK function to request city service."""
    print(f"\n [DEBUG] MOCK CALL: request_city_service Area: {service_area}, Service: '{service_description}'")
    return {"status": "requested", "service_id": "CITY-SVC-001", "message": "City service requested."}

# Passenger Coordinator Agent Mocks (Renamed Citizen Services Agent Mocks for clarity)
def get_citizen_request_status(request_id: str):
    """MOCK function to retrieve citizen request status."""
    print(f"\n [DEBUG] MOCK CALL: get_citizen_request_status Request: {request_id}")
    if request_id == "REQ-001":
        return {"status": "success", "request_id": request_id, "details": "Pothole repair request, status: in progress.", "message": "Request status retrieved."}
    return {"status": "not_found", "message": "Request not found."}

def reroute_service_request(request_id: str, current_department: str, new_department: str):
    """MOCK function to reroute a service request."""
    print(f"\n [DEBUG] MOCK CALL: reroute_service_request Request: {request_id}, Old Dept: {current_department}, New Dept: {new_department}")
    return {"status": "rerouted", "request_id": request_id, "new_department": new_department, "message": "Service request rerouted successfully."}

def send_public_notification(recipient_id: str, message_content: str, contact_method: str):
    """MOCK function to send public notification."""
    print(f"\n [DEBUG] MOCK CALL: send_public_notification Recipient: {recipient_id}, Content: '{message_content}', Method: {contact_method}")
    return {"status": "sent", "recipient_id": recipient_id, "message": "Notification sent successfully."}

# Crew Coordinator Agent Mocks (Renamed Emergency Services Coordinator Agent Mocks)
def check_response_legality(unit_id: str):
    """MOCK function to check response unit legality."""
    print(f"\n [DEBUG] MOCK CALL: check_response_legality Unit: {unit_id}")
    if unit_id == "FireTruck-001":
        return {"status": "compliant", "message": "Emergency unit is compliant for duty."}
    return {"status": "non_compliant", "message": "Emergency unit not compliant or requires review."}

def find_available_unit(role: str, required_qualifications: list, area: str = None):
    """MOCK function to find available units."""
    print(f"\n [DEBUG] MOCK CALL: find_available_unit Role: {role}, Qualifications: {required_qualifications}, Area: {area}")
    if role == "Fire" and "HAZMAT certified" in required_qualifications:
        return {"status": "success", "units_found": ["FireTruck-002", "FireTruck-003"], "message": "Available qualified units found."}
    return {"status": "not_found", "message": "No matching units found."}

def assign_unit_to_incident(unit_id: str, incident_details: str):
    """MOCK function to assign unit to incident."""
    print(f"\n [DEBUG] MOCK CALL: assign_unit_to_incident Unit: {unit_id}, Incident: '{incident_details}'")
    return {"status": "assigned", "unit_id": unit_id, "incident_details": incident_details, "message": "Unit assigned to incident."}

# Flight Dispatcher Agent Mocks (Renamed Smart Grid Operations Agent Mocks)
def monitor_grid_status(component_id: str):
    """MOCK function to monitor grid status."""
    print(f"\n [DEBUG] MOCK CALL: monitor_grid_status Component: {component_id}")
    if component_id == "Substation-A":
        return {"status": "operational", "current_load_mw": 50, "voltage_kv": 110, "message": "Substation-A operating normally."}
    return {"status": "unknown", "message": "Grid component not found or status unknown."}

def analyze_weather_impact_on_energy(area: str, time_window: str):
    """MOCK function to analyze weather impact on energy."""
    print(f"\n [DEBUG] MOCK CALL: analyze_weather_impact_on_energy Area: '{area}', Time Window: {time_window}")
    if "solar farms" in area and "sunny" in time_window:
        return {"status": "positive", "impact": "Increased solar generation", "details": "Expect 20% increase in solar output.", "message": "Positive weather impact."}
    return {"status": "neutral", "impact": "None", "message": "No significant weather impact on energy."}

def calculate_energy_distribution(zone_id: str, current_demand_kwh: float, available_supply_kwh: float, priority_services: list = None):
    """MOCK function to calculate energy distribution."""
    print(f"\n [DEBUG] MOCK CALL: calculate_energy_distribution Zone: {zone_id}, Demand: {current_demand_kwh}kWh, Supply: {available_supply_kwh}kWh, Priority: {priority_services}")
    if available_supply_kwh >= current_demand_kwh:
        return {"status": "balanced", "allocated_kwh": current_demand_kwh, "message": "Energy supply meets demand."}
    else:
        deficit = current_demand_kwh - available_supply_kwh
        return {"status": "deficit", "allocated_kwh": available_supply_kwh, "shortfall_kwh": deficit, "message": f"Energy deficit: {deficit:.2f} kWh. Consider load shedding."}

def issue_grid_directive(component_id: str, directive_summary: str):
    """MOCK function to issue grid directive."""
    print(f"\n [DEBUG] MOCK CALL: issue_grid_directive Component: {component_id}, Directive: '{directive_summary}'")
    return {"status": "issued", "component_id": component_id, "directive": directive_summary, "message": "Grid directive issued."}

# Budget Finance Agent Mocks (Renamed Urban Budget & Finance Agent Mocks)
def get_departmental_budget_status(department: str, period: str):
    """MOCK function to retrieve departmental budget status."""
    print(f"\n [DEBUG] MOCK CALL: get_departmental_budget_status Department: {department}, Period: {period}")
    if department == "Public Works" and period == "Q3 2025":
        return {"status": "success", "department": department, "period": period, "allocated_budget": 10000000.0, "spent_to_date": 7000000.0, "remaining_budget": 3000000.0, "message": "Budget status retrieved."}
    return {"status": "not_found", "message": "Budget status not found for the specified department and period."}

def log_city_expense(expense_category: str, amount: float, project_id: str = None):
    """MOCK function to log a city expense."""
    print(f"\n [DEBUG] MOCK CALL: log_city_expense Category: {expense_category}, Amount: {amount}, Project: {project_id}")
    return {"status": "logged", "expense_id": f"EXP-CITY-{int(time.time())}", "category": expense_category, "amount": amount, "project_id": project_id, "message": "Expense logged successfully."}

def forecast_city_costs(cost_type: str, time_horizon: str):
    """MOCK function to forecast operational costs."""
    print(f"\n [DEBUG] MOCK CALL: forecast_city_costs Cost Type: {cost_type}, Time Horizon: {time_horizon}")
    if cost_type == "utility costs" and time_horizon == "next quarter":
        return {"status": "success", "cost_type": cost_type, "forecasted_amount": 2500000.00, "unit": 'USD', "message": "Utility cost forecast generated."}
    return {"status": "success", "cost_type": cost_type, "forecasted_amount": 0.0, "message": "Forecast not available for this type/horizon."}

def analyze_cost_efficiency(data_json: str, benchmark_data_json: str = None):
    """MOCK function to analyze cost efficiency."""
    print(f"\n [DEBUG] MOCK CALL: analyze_cost_efficiency Data: {data_json}, Benchmark: {benchmark_data_json}")
    try:
        data = json.loads(data_json)
        total_cost = sum(item.get("cost", 0) for item in data)
        efficiency_metric = total_cost / len(data) if data else 0
        return {"status": "success", "total_cost": total_cost, "efficiency_metric": efficiency_metric, "message": "Cost efficiency analysis complete."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for data."}

# Master Tool Executor Mapping (Updated for Smart City Domain)
tool_executor = {
    "design_urban_plan": design_urban_plan,
    "log_project_schedule": log_project_schedule,
    "predict_traffic_congestion": predict_traffic_congestion,
    "optimize_signal_timing": optimize_signal_timing,
    "predict_disruption_impact": predict_disruption_impact,
    "propose_recovery_plan": propose_recovery_plan,
    "assign_resources_to_task": assign_resources_to_task,
    "simulate_resource_deployment": simulate_resource_deployment,
    "schedule_maintenance": schedule_maintenance,
    "check_materials_inventory": check_materials_inventory,
    "log_maintenance_record": log_maintenance_record,
    "get_city_data": get_city_data,
    "analyze_urban_efficiency": analyze_urban_efficiency,
    "check_city_regulation": check_city_regulation,
    "submit_public_safety_report": submit_public_safety_report,
    "analyze_incident": analyze_incident,
    "predict_cascading_effects": predict_cascading_effects,
    "track_city_activity": track_city_activity,
    "request_city_service": request_city_service,
    "get_citizen_request_status": get_citizen_request_status,
    "reroute_service_request": reroute_service_request,
    "send_public_notification": send_public_notification,
    "check_response_legality": check_response_legality,
    "find_available_unit": find_available_unit,
    "assign_unit_to_incident": assign_unit_to_incident,
    "monitor_grid_status": monitor_grid_status,
    "analyze_weather_impact_on_energy": analyze_weather_impact_on_energy,
    "calculate_energy_distribution": calculate_energy_distribution,
    "issue_grid_directive": issue_grid_directive,
    "get_departmental_budget_status": get_departmental_budget_status,
    "log_city_expense": log_city_expense,
    "forecast_city_costs": forecast_city_costs,
    "analyze_cost_efficiency": analyze_cost_efficiency,
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: General smart city information."
}

# Function to standardize tools for client.chat.complete (remains the same)
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# Test Case Execution for Smart City Agents (Adapted from AOCC Test Cases)
print("\n--- Executing Test Cases for Smart City Agents (via chat completions) ---")

test_cases = [
    {
        "agent": urban_planning_agent,
        "name": "Urban Planning & Development Agent",
        "query": "Design a new residential zone in the downtown area, considering current zoning laws.",
        "expected_tool_call": "design_urban_plan"
    },
    {
        "agent": urban_traffic_management_agent,
        "name": "Urban Traffic Management Agent",
        "query": "Predict traffic congestion at the intersection of Main Street and Elm Avenue for the next 2 hours.",
        "expected_tool_call": "predict_traffic_congestion"
    },
    {
        "agent": urban_disruption_response_agent,
        "name": "Urban Disruption Response Agent",
        "query": "Predict impact for downtown due to a major water pipe burst.",
        "expected_tool_call": "predict_disruption_impact"
    },
    {
        "agent": city_resource_allocation_agent,
        "name": "City Resource Allocation Agent",
        "query": "Assign emergency response team 'ERT-Alpha' and vehicle 'Truck-005' to the incident at Oak Street.",
        "expected_tool_call": "assign_resources_to_task"
    },
    {
        "agent": infrastructure_maintenance_agent,
        "name": "Infrastructure Maintenance Agent",
        "query": "Schedule urgent repair for the bridge on Maple Avenue on 2025-08-01.",
        "expected_tool_call": "schedule_maintenance"
    },
    {
        "agent": urban_data_analysis_agent,
        "name": "Urban Data Analytics Agent",
        "query": "Get energy consumption data for the downtown district from 'smart meters' for June 2025.",
        "expected_tool_call": "get_city_data"
    },
    {
        "agent": city_compliance_safety_agent,
        "name": "City Compliance & Safety Agent",
        "query": "Check building code compliance for new high-rise construction in the Financial District.",
        "expected_tool_call": "check_city_regulation"
    },
    {
        "agent": urban_incident_response_agent,
        "name": "Urban Incident Response Agent",
        "query": "Analyze incident UINC-005 using operational data: {\"sensor_readings\": \"traffic sensor anomaly\", \"location\": \"MainSt_ElmAve\"}.",
        "expected_tool_call": "analyze_incident"
    },
    {
        "agent": public_works_coordination_agent,
        "name": "Public Works Coordination Agent",
        "query": "Track street cleaning for the downtown core.",
        "expected_tool_call": "track_city_activity"
    },
    {
        "agent": citizen_services_agent,
        "name": "Citizen Services Agent",
        "query": "Get status for citizen request REQ-001.",
        "expected_tool_call": "get_citizen_request_status"
    },
    {
        "agent": citizen_services_agent,
        "name": "Citizen Services Agent",
        "query": "Send notification to residents of District 3 regarding scheduled road closures.",
        "expected_tool_call": "send_public_notification"
    },
    {
        "agent": emergency_services_coordinator_agent,
        "name": "Emergency Services Coordinator Agent",
        "query": "Check legality for FireTruck-001.",
        "expected_tool_call": "check_response_legality"
    },
    {
        "agent": emergency_services_coordinator_agent,
        "name": "Emergency Services Coordinator Agent",
        "query": "Find available Fire units with HAZMAT certified qualification in the East Zone.",
        "expected_tool_call": "find_available_unit"
    },
    {
        "agent": smart_grid_operations_agent,
        "name": "Smart Grid Operations Agent",
        "query": "Monitor current power load in Substation-A.",
        "expected_tool_call": "monitor_grid_status"
    },
    {
        "agent": smart_grid_operations_agent,
        "name": "Smart Grid Operations Agent",
        "query": "Analyze weather impact on energy generation for solar farms in the city for the next 24 hours.",
        "expected_tool_call": "analyze_weather_impact_on_energy"
    },
    {
        "agent": smart_grid_operations_agent,
        "name": "Smart Grid Operations Agent",
        "query": "Calculate energy distribution for Zone-C, current demand 10000 kWh, available supply 12000 kWh, prioritize hospitals.",
        "expected_tool_call": "calculate_energy_distribution"
    },
    {
        "agent": urban_budget_finance_agent,
        "name": "Urban Budget & Finance Agent",
        "query": "Get budget status for Public Works Department for Q3 2025.",
        "expected_tool_call": "get_departmental_budget_status"
    },
    {
        "agent": urban_budget_finance_agent,
        "name": "Urban Budget & Finance Agent",
        "query": "Log an infrastructure maintenance expense of 50000 for the 'Bridge Repair Project'.",
        "expected_tool_call": "log_city_expense"
    },
    {
        "agent": urban_budget_finance_agent,
        "name": "Urban Budget & Finance Agent",
        "query": "Forecast utility costs for the next quarter.",
        "expected_tool_call": "forecast_city_costs"
    },
    {
        "agent": urban_budget_finance_agent,
        "name": "Urban Budget & Finance Agent",
        "query": "Analyze cost efficiency for recent public safety operations with data: [{\"item\": \"police patrol\", \"cost\": 10000}, {\"item\": \"fire response\", \"cost\": 15000}].",
        "expected_tool_call": "analyze_cost_efficiency"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"]

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")
    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f" [DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f"  Tool Name: {tool_call.function.name}")
                print(f"  Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None

                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")

                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call requested.
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the context.
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)

            # Add the final assistant response to history (optional for a single-turn demo, but good practice).
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)
        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")

Mounted at /content/gdrive
Creating AI agents for Smart City Domain...
Urban Planning & Development Agent 'urban-planning-agent' created with ID: ag_0685e8efef50714680000e8d2077371b
Urban Traffic Management Agent 'urban-traffic-agent' created with ID: ag_0685e8eff13e7d2580007b1fe3131b7a
Urban Disruption Response Agent 'urban-disruption-agent' created with ID: ag_0685e8eff3217287800054b3635d05b9
City Resource Allocation Agent 'city-resource-agent' created with ID: ag_0685e8eff4787cf58000150758603ce1
Infrastructure Maintenance Agent 'infrastructure-maintenance-agent' created with ID: ag_0685e8eff59a737e8000bb9f859ea319
Urban Data Analytics Agent 'urban-data-analysis-agent' created with ID: ag_0685e8eff7d875768000c84c5b57122b
City Compliance & Safety Agent 'city-compliance-safety-agent' created with ID: ag_0685e8eff9807fd38000edbacc4d66c5
Urban Incident Response Agent 'urban-incident-response-agent' created with ID: ag_0685e8effbb874a280005395ea9707a7
Public Works Coordination Agent 'publ